## Introducción

Una red neuronal artificial (RNA) preentrenada es un modelo que ha sido entrenado previamente y luego distribuído (incluyendo arquitectura y valores de pesos) para su uso posterior. De esta manera no es necesario que volver a entrenarlo, o bien, se puede utilizar parte de el ya entrenado, y entrenar otras partes, realizando un ajuste fino o fine tuning. Constituye una manera de reducir los recursos y tiempos necesarios para generar un sistema de aprendizaje automático.

Hugging Face es una plataforma de aprendizaje automático, conocida por su librería de [Transformers](https://es.wikipedia.org/wiki/Transformador(modelo_de_aprendizajeautom%C3%A1tico)) creada para aplicaciones de procesamiento de lenguaje natural y que permite a los usuarios compartir conjuntos de datos y modelos de aprendizaje automático preentrenados. Su Model Hub contiene miles de modelos previamente entrenados de código abierto que cualquiera puede descargar y usar. Además dispone de varios cursos en los que se enseñan conceptos de aprendizaje automático a la vez que como utilizar sus herramientas.

Una de las formas más básicas de usar para inferencia la librería Hugging Face Transformers es la función pipeline(), que permite conectar uno de sus modelos de con los pasos necesarios para su preprocesamiento y posprocesamiento, abstrayendo la mayor parte del código complejo de la librería y ofreciendo una API simple dedicada a varias tareas, entre ellas, audio, visión artificial, procesamiento de lenguaje natural y tareas multimodales.

In [1]:
from transformers import pipeline

# Avoid some warning
%env TOKENIZERS_PARALLELISM=false

2025-01-09 08:54:49.552576: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 08:54:49.552609: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 08:54:49.553566: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 08:54:49.558807: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-09 08:54:50.110308: W tensorflow/compiler/tf2

env: TOKENIZERS_PARALLELISM=false


## Análisis de sentimientos
El análisis de sentimientos premite clasificar textos de acuerdo a distintas etiquetas predeterminadas, como podrían ser textos positivos frente a negativos. Existen disintos modelos preentrenados disponibles para ello y para distintos idiomas, por ejemplo Spanish Sentiment Analysis Classifier

In [2]:
# Allocate a piepeline for spanish sentiment-analysis
classifier = pipeline(model='VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis')
text_1 = "Nos alegra introducirles las redes neuronales artificiales preentrenadas a ustedes."
classifier(text_1)

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'Positive', 'score': 0.9995808005332947}]

In [3]:
text_2 = "Sería una lástima que el tema no les resultara interesante."
classifier(text_2)

[{'label': 'Negative', 'score': 0.9994773268699646}]

In [4]:
# Showing the classifier model
print(classifier.model)
# classifier.model.summary()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

... o también podemos realizar puntuaciones sobre los textos usando nlptown/bert-base-multilingual-uncased-sentiment.

In [5]:
# Ranking texts
classifier = pipeline(model='nlptown/bert-base-multilingual-uncased-sentiment')
print(classifier(text_1))
print(classifier(text_2))

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': '3 stars', 'score': 0.2429007887840271}]
[{'label': '2 stars', 'score': 0.44524437189102173}]


Podemos traducir los textos a inglés mediante Helsinki-NLP/opus-mt-es-en y realizar las puntuaciones nuevamente.

In [6]:
# Translate the texts
translator = pipeline(model='Helsinki-NLP/opus-mt-es-en')
text_1_translated = translator(text_1)[0]['translation_text']
text_2_translated = translator(text_2)[0]['translation_text']

# And rank again
print(text_1_translated, classifier(text_1_translated))
print(text_2_translated, classifier(text_2_translated))

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


We're glad to introduce the artificial neural networks pre-trained to you. [{'label': '5 stars', 'score': 0.580280065536499}]
It would be a shame if they didn't find the subject interesting. [{'label': '3 stars', 'score': 0.3762303590774536}]


In [ ]:
# !nvidia-smi

# Reiniciando kernel para limpiar la memoria de la GPU
!pkill -9 -f ipykernel_launcher